In [3]:
# access for combinded / cleaned data
%run CleanData.ipynb

c:\Users\evani\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
X_train.shape

(99195, 133)

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Masking
from tensorflow.keras.utils import to_categorical

# Reshape input for RNN: (samples, timesteps, features)
# Here, treat each row as a sequence of 188 timesteps with 1 feature per timestep
X_train_rnn = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_rnn = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

# Convert labels to categorical if needed
num_classes = int(max(y_train.max(), y_test.max())) + 1
y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_test_cat = to_categorical(y_test, num_classes=num_classes)

# Build RNN model
model = Sequential([
    Masking(mask_value=0., input_shape=(X_train.shape[1], 1)),
    SimpleRNN(32, activation='tanh'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_rnn, y_train_cat, epochs=10, batch_size=64, validation_data=(X_test_rnn, y_test_cat))



Epoch 1/10


C:\Users\evani\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\masking.py:48: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1550/1550 ━━━━━━━━━━━━━━━━━━━━ 26s 16ms/step - accuracy: 0.7295 - loss: 1.0321 - val_accuracy: 0.7337 - val_loss: 0.8978
Epoch 2/10
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 25s 16ms/step - accuracy: 0.7582 - loss: 0.8419 - val_accuracy: 0.7706 - val_loss: 0.7806
Epoch 3/10
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 26s 17ms/step - accuracy: 0.7728 - loss: 0.7875 - val_accuracy: 0.7665 - val_loss: 0.8462
Epoch 4/10
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 26s 17ms/step - accuracy: 0.7811 - loss: 0.7398 - val_accuracy: 0.7883 - val_loss: 0.6967
Epoch 5/10
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 26s 17ms/step - accuracy: 0.7899 - loss: 0.6869 - val_accuracy: 0.7970 - val_loss: 0.6570
Epoch 6/10
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 26s 17ms/step - accuracy: 0.7965 - loss: 0.6632 - val_accuracy: 0.7920 - val_loss: 0.7022
Epoch 7/10
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 26s 17ms/step - accuracy: 0.7791 - loss: 0.7623 - val_accuracy: 0.7685 - val_loss: 0.8699
Epoch 8/10
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 27s 17ms/step - accuracy: 0.7737 - loss: 0.83

In [6]:
history.model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 133, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 32)             │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │           264 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,058 (15.86 KB)

 Trainable params: 1,352 (5.28 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,706 (10.57 KB)

In [7]:
from tensorflow.keras.layers import LSTM

# Reshape input for LSTM: (samples, timesteps, features)
X_train_lstm = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_lstm = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

# Build LSTM model
lstm_model = Sequential([
    Masking(mask_value=0., input_shape=(X_train.shape[1], 1)),
    LSTM(32, activation='tanh'),
    Dense(num_classes, activation='softmax')
])

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the LSTM model
lstm_history = lstm_model.fit(X_train_lstm, y_train_cat, epochs=10, batch_size=64, validation_data=(X_test_lstm, y_test_cat))

Epoch 1/10
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 55s 34ms/step - accuracy: 0.7224 - loss: 1.0807 - val_accuracy: 0.7305 - val_loss: 1.0092
Epoch 2/10
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 55s 36ms/step - accuracy: 0.7305 - loss: 1.0073 - val_accuracy: 0.7305 - val_loss: 0.9889
Epoch 3/10
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 60s 39ms/step - accuracy: 0.7318 - loss: 0.9723 - val_accuracy: 0.7485 - val_loss: 0.8416
Epoch 4/10
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 60s 39ms/step - accuracy: 0.7537 - loss: 0.8171 - val_accuracy: 0.7672 - val_loss: 0.7552
Epoch 5/10
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 61s 39ms/step - accuracy: 0.7685 - loss: 0.7511 - val_accuracy: 0.7823 - val_loss: 0.7204
Epoch 6/10
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 60s 39ms/step - accuracy: 0.7888 - loss: 0.7017 - val_accuracy: 0.8045 - val_loss: 0.6621
Epoch 7/10
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 60s 38ms/step - accuracy: 0.8012 - loss: 0.6609 - val_accuracy: 0.7997 - val_loss: 0.6507
Epoch 8/10
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 60s 39ms/step - accuracy: 0.8146 -

In [8]:
history.model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 133, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 32)             │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │           264 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,058 (15.86 KB)

 Trainable params: 1,352 (5.28 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,706 (10.57 KB)

In [12]:
loss, accuracy = lstm_model.evaluate(X_test, y_test_cat)
print(f"Test Accuracy: {accuracy:.4f}")

ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=False
  • mask=None